In [2]:
import delf as df

In [3]:
import sys
import os.path as path

modulos_path = path.abspath('../minIA')
if modulos_path not in sys.path:
    sys.path.append(modulos_path)

from utiles import lectura_img
import numpy as np

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys
import time

import numpy as np
from six.moves import range
import tensorflow as tf

from google.protobuf import text_format
from tensorflow.python.platform import app
from delf import delf_config_pb2
from delf import feature_io
from delf import utils
from delf import extractor

https://github.com/tensorflow/models/blob/master/research/delf/delf/python/examples/extract_features.py

# Original example

In [4]:
def main(image_paths, config_path, output_dir):
  # Read list of images.
  print('Reading list of images...')
  num_images = len(image_paths)
  print(f'done! Found {num_images} images')

  # Parse DelfConfig proto.
  config = delf_config_pb2.DelfConfig()
  with tf.io.gfile.GFile(config_path, 'r') as f:
    text_format.Merge(f.read(), config)

  # Create output directory if necessary.
  if not tf.io.gfile.exists(output_dir):
    tf.io.gfile.makedirs(output_dir)

  extractor_fn = extractor.MakeExtractor(config)

  start = time.time()
  for i in range(num_images):
    # Report progress once in a while.
    if i == 0:
      print('Starting to extract DELF features from images...')
    elif i % _STATUS_CHECK_ITERATIONS == 0:
      elapsed = (time.time() - start)
      print(
          f'Processing image {i} out of {num_images}, last '
          f'{_STATUS_CHECK_ITERATIONS} images took {elapsed} seconds'
      )
      start = time.time()

    # If descriptor already exists, skip its computation.
    out_desc_filename = os.path.splitext(os.path.basename(
        image_paths[i]))[0] + _DELF_EXT
    out_desc_fullpath = os.path.join(output_dir, out_desc_filename)
    if tf.io.gfile.exists(out_desc_fullpath):
      print(f'Skipping {image_paths[i]}')
      continue

    im = np.array(utils.RgbLoader(image_paths[i]))

    # Extract and save features.
    extracted_features = extractor_fn(im)
    locations_out = extracted_features['local_features']['locations']
    descriptors_out = extracted_features['local_features']['descriptors']
    feature_scales_out = extracted_features['local_features']['scales']
    attention_out = extracted_features['local_features']['attention']

    feature_io.WriteToFile(out_desc_fullpath, locations_out, feature_scales_out,
                           descriptors_out, attention_out)
    return descriptors_out

In [7]:
# Extension of feature files.
_DELF_EXT = '.delf'
# Pace to report extraction log.
_STATUS_CHECK_ITERATIONS = 100

image_paths = lectura_img('D:\Archivos\Proyectos\minIA\images\GrupoDELF')
config_path = 'D:\Archivos\Proyectos\minIA\\notebooks\delf_config_example.pbtxt'
output_dir = 'D:\Archivos\Proyectos\minIA\descriptors\delf'
descrip = main(image_paths, config_path,output_dir)

Reading list of images...
done! Found 36 images
Starting to extract DELF features from images...
Skipping D:\Archivos\Proyectos\minIA\images\GrupoDELF\100008.jpg
Skipping D:\Archivos\Proyectos\minIA\images\GrupoDELF\100023.jpg


## Modificación

In [5]:
import pickle


In [6]:
def extractorDELF(image_paths, config_path, output_dir):
  # Read list of images.
  print('Reading list of images...')
  num_images = len(image_paths)
  print(f'done! Found {num_images} images')

  # Parse DelfConfig proto.
  config = delf_config_pb2.DelfConfig()
  with tf.io.gfile.GFile(config_path, 'r') as f:
    text_format.Merge(f.read(), config)

  # Create output directory if necessary.
  if not tf.io.gfile.exists(output_dir):
    tf.io.gfile.makedirs(output_dir)
    
  #Create pickle file
  path_pickle = path.abspath(output_dir+'_DELF.pickle')
  pickle_file = open(path_pickle, 'wb')

  #Create lista de descriptor por imagen
  descriptors = list()

  extractor_fn = extractor.MakeExtractor(config)

  start = time.time()
  for i in range(num_images):
    # Report progress once in a while.
    if i == 0:
      print('Starting to extract DELF features from images...')
    elif i % _STATUS_CHECK_ITERATIONS == 0:
      elapsed = (time.time() - start)
      print(
          f'Processing image {i} out of {num_images}, last '
          f'{_STATUS_CHECK_ITERATIONS} images took {elapsed} seconds'
      )
      start = time.time()

    im = np.array(utils.RgbLoader(image_paths[i]))

    # Extract and save features.
    extracted_features = extractor_fn(im)
    descriptors.append(extracted_features['local_features'])
  pickle.dump(descriptors, pickle_file)
  print("Termino")
  return descriptors

In [7]:
#Output incluye el nombre del archivo

# Pace to report extraction log.
_STATUS_CHECK_ITERATIONS = 100

image_paths = lectura_img('D:\Archivos\Proyectos\minIA\images\GrupoDELF')
config_path = 'D:\Archivos\Proyectos\minIA\\notebooks\delf_config_example.pbtxt'
output_dir = 'D:\Archivos\Proyectos\minIA\descriptors\delf\PruebaD1'
descrip = extractorDELF(image_paths, config_path,output_dir)

Reading list of images...
done! Found 36 images


Starting to extract DELF features from images...
Termino


In [14]:
print(type(descrip[0]['descriptors']))
print(len(descrip[0]['descriptors']))

print(len(descrip[0]['descriptors'][0]))

<class 'numpy.ndarray'>
20
40


In [25]:
print (descrip[1]['locations'])
print (descrip[1]['scales'])

[[128.       128.      ]
 [128.        64.      ]
 [176.       144.      ]
 [113.138176 181.02109 ]
 [224.        16.      ]
 [128.         0.      ]
 [ 64.        32.      ]
 [180.99548  180.99548 ]
 [112.        96.      ]
 [192.       144.      ]
 [208.        16.      ]
 [ 90.49774   90.49774 ]
 [112.       192.      ]
 [226.27635   22.627632]
 [ 45.255264 226.27635 ]
 [ 90.49774  180.99548 ]
 [ 90.510544  90.510544]
 [203.64871   22.627632]
 [ 90.510544 226.27635 ]
 [113.138176 203.64871 ]
 [ 64.        16.      ]
 [113.138176 135.76581 ]
 [112.       144.      ]
 [128.       192.      ]
 [128.       192.      ]
 [224.        32.      ]
 [  0.       128.      ]
 [ 96.        64.      ]
 [224.       240.      ]
 [ 64.         0.      ]
 [ 90.510544 181.02107 ]
 [226.27635   45.255264]
 [160.       144.      ]
 [181.02107    0.      ]
 [181.02109  135.76581 ]
 [ 16.         0.      ]
 [ 80.        64.      ]
 [192.        32.      ]
 [ 64.       224.      ]
 [ 80.       224.      ]


skimage import feature